In [3]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn import metrics
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')



from sklearn.model_selection import cross_val_score

import vectorize_functions


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\fried\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Einladen der vektorisierten Daten

In [4]:
nltk.download('punkt')
X_train, X_test, y_train, y_test = vectorize_functions.vectorize_w2v()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fried\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
positive = np.count_nonzero(y_train == 1)
negative = np.count_nonzero(y_train == 0)
print("Train")
print("- Positive:", positive)
print("- Negative:", negative)
print("- Verhältnis:", negative / positive)

positive = np.count_nonzero(y_test == 1)
negative = np.count_nonzero(y_test == 0)
print("Test")
print("- Positive:", positive)
print("- Negative:", negative)
print("- Verhältnis:", negative / positive)

Train
- Positive: 1272
- Negative: 18055
- Verhältnis: 14.194182389937106
Test
- Positive: 533
- Negative: 7750
- Verhältnis: 14.540337711069418


In [6]:
print("X_train shape", X_train.shape)
print("y_train shape", y_train.shape)

print("X_test shape", X_test.shape)
print("y_test shape", y_test.shape)

X_train shape (19327, 100)
y_train shape (19327,)
X_test shape (8283, 100)
y_test shape (8283,)


# Evaluation

In [7]:
evaluation = pd.DataFrame(
    columns=["model", "variant", "train_acc", "train_prec", "train_rec", "train_f1", "test_acc", "test_prec",
             "test_rec", "test_f1"])

In [8]:
def add_to_eval_df(model, model_name, variant, x_data_train, y_data_train, x_data_test, y_data_test):
    train_acc = model.score(x_data_train, y_data_train)
    train_precision = precision_score(y_data_train, model.predict(x_data_train))
    train_recall = recall_score(y_data_train, model.predict(x_data_train))
    train_f1 = f1_score(y_data_train, model.predict(x_data_train))

    test_acc = model.score(x_data_test, y_data_test)
    test_precision = precision_score(y_data_test, model.predict(x_data_test))
    test_recall = recall_score(y_data_test, model.predict(x_data_test))
    test_f1 = f1_score(y_data_test, model.predict(x_data_test))

    evaluation.loc[len(evaluation.index)] = [model_name, variant, train_acc, train_precision, train_recall, train_f1,
                                             test_acc, test_precision, test_recall, test_f1]

In [10]:
def evaluate_model(model, x_test, y_test, sampling_method):
    pred = model.predict(x_test)
    accscore = metrics.accuracy_score(pred, y_test)

    print(f'{sampling_method} model accuracy for classification is =', str('{:04.2f}'.format(accscore * 100)) + '%')
    print('------------------------------------------------')
    print('Confusion Matrix:')
    print(pd.DataFrame(confusion_matrix(y_test, pred)))
    print('------------------------------------------------')
    print('Classification Report:')
    print(classification_report(y_test, pred))

    # probs = model.predict_proba(x_test)
    # preds = probs[:, 1]
    # fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
    # roc_auc = metrics.auc(fpr, tpr)

    # plt.title('Receiver Operating Characteristic')
    # plt.plot(fpr, tpr, 'b', label='AUC = %0.2f' % roc_auc)
    # plt.legend(loc='lower right')
    # plt.plot([0, 1], [0, 1], 'r--')
    # plt.xlim([0, 1])
    # plt.ylim([0, 1])
    # plt.ylabel('True Positive Rate')
    # plt.xlabel('False Positive Rate')
    # plt.show()


# SVM


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

results_list = []

param_grid = {
    'kernel': ['linear', 'sigmoid'],
    'degree': [3, 4, 5],
    'C': [0.01, 0.1, 1],
    'gamma': [0.01, 0.1, 1],
    'random_state': [42],
    'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3}]
}

svm = SVC()

grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, scoring='f1', cv=3, verbose=2, n_jobs=-1)

grid_search.fit(X_train, y_train)

for params in grid_search.cv_results_['params']:
    best_model = grid_search.best_estimator_

    y_train_pred = best_model.predict(X_train)

    y_test_pred = best_model.predict(X_test)

    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_recall = recall_score(y_train, y_train_pred)
    train_precision = precision_score(y_train, y_train_pred)
    train_f1 = f1_score(y_train, y_train_pred)

    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_recall = recall_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)

    result_dict = {
        'kernel': params['kernel'],
        'degree': params['degree'],
        'C': params['C'],
        'gamma': params['gamma'],
        'class_weight': params['class_weight'],
        'train_accuracy': train_accuracy,
        'train_recall': train_recall,
        'train_precision': train_precision,
        'train_f1': train_f1,
        'test_accuracy': test_accuracy,
        'test_recall': test_recall,
        'test_precision': test_precision,
        'test_f1': test_f1
    }

    results_list.append(result_dict)

results_df = pd.DataFrame(results_list)

results_df.to_csv('svm_grid_w2v_lin_sig.csv', index=False)

print(results_df)


Fitting 3 folds for each of 108 candidates, totalling 324 fits
      kernel  degree     C  gamma  class_weight  train_accuracy  train_recall  \
0     linear       3  0.01   0.01  {0: 1, 1: 1}        0.922233      0.022013   
1    sigmoid       3  0.01   0.01  {0: 1, 1: 1}        0.922233      0.022013   
2     linear       3  0.01   0.10  {0: 1, 1: 1}        0.922233      0.022013   
3    sigmoid       3  0.01   0.10  {0: 1, 1: 1}        0.922233      0.022013   
4     linear       3  0.01   1.00  {0: 1, 1: 1}        0.922233      0.022013   
..       ...     ...   ...    ...           ...             ...           ...   
103  sigmoid       5  1.00   0.01  {0: 1, 1: 3}        0.922233      0.022013   
104   linear       5  1.00   0.10  {0: 1, 1: 3}        0.922233      0.022013   
105  sigmoid       5  1.00   0.10  {0: 1, 1: 3}        0.922233      0.022013   
106   linear       5  1.00   1.00  {0: 1, 1: 3}        0.922233      0.022013   
107  sigmoid       5  1.00   1.00  {0: 1, 1: 3

Fitting 3 folds for each of 108 candidates, totalling 324 fits


    kernel  degree     C  gamma  class_weight  train_accuracy  train_recall  \
0      rbf       3  0.01   0.01  {0: 1, 1: 1}        0.933254       0.02673   
1     poly       3  0.01   0.01  {0: 1, 1: 1}        0.933254       0.02673   
2      rbf       3  0.01   0.10  {0: 1, 1: 1}        0.933254       0.02673   
3     poly       3  0.01   0.10  {0: 1, 1: 1}        0.933254       0.02673   
4      rbf       3  0.01   1.00  {0: 1, 1: 1}        0.933254       0.02673   
..     ...     ...   ...    ...           ...             ...           ...   
103   poly       5  1.00   0.01  {0: 1, 1: 3}        0.933254       0.02673   
104    rbf       5  1.00   0.10  {0: 1, 1: 3}        0.933254       0.02673   
105   poly       5  1.00   0.10  {0: 1, 1: 3}        0.933254       0.02673   
106    rbf       5  1.00   1.00  {0: 1, 1: 3}        0.933254       0.02673   
107   poly       5  1.00   1.00  {0: 1, 1: 3}        0.933254       0.02673   

     train_precision  train_f1  test_accuracy  test